In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

In [2]:
#load data
#this might take a while as it will download the dataset from internet
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
data_train = torchvision.datasets.MNIST('./', download=True, train=True, transform = transform)
data_test = torchvision.datasets.MNIST('./', download=True, train=False, transform = transform)


Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw




Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw




Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw




Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw

Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [6]:
# DATA LOADER

from torch.utils.data import DataLoader
# batch size of 30 or 60ish
data_train_loader = DataLoader(data_train, batch_size=64, shuffle=True)
data_test_loader = DataLoader(data_test, batch_size=64, shuffle=False)

# for image, label in data_train_loader:
#     print(f"Image: {image}\tLabel: {label}")

In [4]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 128) 
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10) 
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.log_softmax(self.fc3(x), dim=1)
        return x
    

        

In [5]:
criterion = nn.NLLLoss() # Negative Log Likelyhood Loss


In [12]:
network = Net()

optimizer = optim.SGD(network.parameters(), lr=0.005, momentum=0.9) # implements stochastic gradient descent
    
for epoch in range(5):
    loss_train = 0
    # network.train()

    for images, labels in data_train_loader:
        optimizer.zero_grad() # zeros out the gradients (i.e. resets the gradient)
        images = images.view(images.shape[0], -1) # setting up images to be the correct shape
        output = network(images)

        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        loss_train += loss.item()
    print(f"{epoch} Loss Train: {loss_train / len(data_train_loader)}")
    
    # network.eval()
    with torch.no_grad():
        loss_test = 0
        for images, labels in data_test_loader:
            optimizer.zero_grad()
            images = images.view(images.shape[0], -1)
            output = network(images)
            loss = criterion(output, labels)
            # loss.backward()
            # optimizer.step()
            loss_test += loss.item()
    
        print(f"{epoch} Loss Test: {loss_test / len(data_test_loader)}")
        
        

0 Loss Train: 0.5215176009912608
0 Loss Test: 0.31189382758442386
1 Loss Train: 0.2331892415277485
1 Loss Test: 0.17999759385552092
2 Loss Train: 0.16788425382329966
2 Loss Test: 0.1487057150021243
3 Loss Train: 0.13374581217948497
3 Loss Test: 0.12567644951432042
4 Loss Train: 0.11080441919705475
4 Loss Test: 0.10613289674310974
